In [5]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from afinn import Afinn
from pattern.en import sentiment
nltk.download('vader_lexicon')
nltk.download('punkt')
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.layers import BatchNormalization
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from keras.layers import Embedding, GRU, Dense, concatenate, Input
from keras.regularizers import l1_l2
from keras.models import Sequential
import tensorflow.compat.v1 as tf

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\maddy\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maddy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
df=pd.read_csv('truthseeker_emotions.csv')

In [7]:
df.head(3)

,Unnamed: 0,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer,anger,disgust,fear,joy,neutral,sadness,surprise
0,0,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@POTUS Biden Blunders - 6 Month Update\n\nInfl...,Mostly Agree,Agree,0,0,0,0,0,1,0
1,1,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,NO MAJORITY,Agree,0,0,0,0,0,1,0
2,2,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",THE SUPREME COURT is siding with super rich pr...,Agree,Agree,0,0,0,0,0,1,0


In [8]:
df['textblob'] = df['statement'].apply(lambda x: TextBlob(x).sentiment.polarity)
afinn = Afinn()
df['afinn'] = df['statement'].apply(lambda x: afinn.score(x))
df['pattern'] = df['statement'].apply(lambda x: sentiment(x)[0])
analyzer = SentimentIntensityAnalyzer()

df['vader'] = df['statement'].apply(lambda x: analyzer.polarity_scores(x)['compound'])


In [9]:
df.head(3)

,Unnamed: 0,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer,anger,disgust,fear,joy,neutral,sadness,surprise,textblob,afinn,pattern,vader
0,0,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@POTUS Biden Blunders - 6 Month Update\n\nInfl...,Mostly Agree,Agree,0,0,0,0,0,1,0,0.0,-1.0,0.0,-0.6908
1,1,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,NO MAJORITY,Agree,0,0,0,0,0,1,0,0.0,-1.0,0.0,-0.6908
2,2,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",THE SUPREME COURT is siding with super rich pr...,Agree,Agree,0,0,0,0,0,1,0,0.0,-1.0,0.0,-0.6908


In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

In [11]:
selected_columns = ['statement', 'BinaryNumTarget', 'anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise',
                    'afinn', 'pattern', 'vader', 'textblob']

In [12]:
data = df[selected_columns]


In [13]:
data

,statement,BinaryNumTarget,anger,disgust,fear,joy,neutral,sadness,surprise,afinn,pattern,vader,textblob
0,End of eviction moratorium means millions of A...,1.0,0,0,0,0,0,1,0,-1.0,0.0,-0.6908,0.0
1,End of eviction moratorium means millions of A...,1.0,0,0,0,0,0,1,0,-1.0,0.0,-0.6908,0.0
2,End of eviction moratorium means millions of A...,1.0,0,0,0,0,0,1,0,-1.0,0.0,-0.6908,0.0
3,End of eviction moratorium means millions of A...,1.0,0,0,0,0,0,1,0,-1.0,0.0,-0.6908,0.0
4,End of eviction moratorium means millions of A...,1.0,0,0,0,0,0,1,0,-1.0,0.0,-0.6908,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134193,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,1,0,0,0,0,0,0,0.0,0.0,-0.3182,0.0
134194,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,1,0,0,0,0,0,0,0.0,0.0,-0.3182,0.0
134195,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,1,0,0,0,0,0,0,0.0,0.0,-0.3182,0.0
134196,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,1,0,0,0,0,0,0,0.0,0.0,-0.3182,0.0


In [14]:
df1=df

In [15]:
duplicates = df1[df1.duplicated(subset=['statement'], keep=False)].sum()

# Display the duplicate rows
print(duplicates)

Unnamed: 0                                                        9002629269
author                     D.L. DavisD.L. DavisD.L. DavisD.L. DavisD.L. D...
statement                  End of eviction moratorium means millions of A...
target                                                                 68911
BinaryNumTarget                                                      68911.0
manual_keywords            Americans, eviction moratoriumAmericans, evict...
tweet                      @POTUS Biden Blunders - 6 Month Update\n\nInfl...
5_label_majority_answer    Mostly AgreeNO MAJORITYAgreeMostly AgreeAgreeA...
3_label_majority_answer    AgreeAgreeAgreeAgreeAgreeAgreeAgreeAgreeAgreeA...
anger                                                                  12384
disgust                                                                16788
fear                                                                    6739
joy                                                                     2825

In [16]:
df1.drop_duplicates(subset=['statement'], inplace=True)


In [17]:
df1['BinaryNumTarget'] = df1['BinaryNumTarget'].astype(int)

In [18]:
emotions_columns = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
df1[emotions_columns] = df1[emotions_columns].astype(float)

In [19]:
df1.dropna(inplace=True)

In [20]:
df1.head(1)

,Unnamed: 0,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer,anger,disgust,fear,joy,neutral,sadness,surprise,textblob,afinn,pattern,vader
0,0,D.L. Davis,End of eviction moratorium means millions of A...,True,1,"Americans, eviction moratorium",@POTUS Biden Blunders - 6 Month Update\n\nInfl...,Mostly Agree,Agree,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,-0.6908


In [21]:
X_text = df1['statement'].values
X_emotions = df1[['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']].values
X_scores = df1[['afinn', 'pattern', 'vader', 'textblob']].values
y = df1['BinaryNumTarget'].values


In [22]:
statements = df1['statement']

max_len_text = max(len(statement.split()) for statement in statements)

print("Maximum length of statements:", max_len_text)

Maximum length of statements: 48


In [23]:
max_len_text = max_len_text  
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_text)
sequences = tokenizer.texts_to_sequences(X_text)
X_text_pad = pad_sequences(sequences, maxlen=max_len_text)

In [24]:
embedding_dim = 100  
vocab_size = len(tokenizer.word_index) + 1


In [25]:
from sklearn.model_selection import StratifiedKFold

In [26]:
def create_model():
    model_text = Sequential()
    model_text.add(Embedding(vocab_size, embedding_dim, input_length=max_len_text))
    model_text.add(Bidirectional(LSTM(128))) 

    model_emotions = Sequential()
    model_emotions.add(Dense(64, input_shape=(7,)))  # 7 features for emotions 
    model_scores = Sequential()
    model_scores.add(Dense(64, input_shape=(4,)))  # 4 features for sentiment scores

    merged = concatenate([model_text.output, model_emotions.output, model_scores.output])
    merged = Dense(64, activation='relu')(merged)
    output = Dense(1, activation='sigmoid')(merged)

    model = tf.keras.models.Model(inputs=[model_text.input, model_emotions.input, model_scores.input], outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [27]:
n_splits = 5  
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)


In [28]:
for fold, (train_index, test_index) in enumerate(skf.split(X_text_pad, y)):
    print(f"Training Fold {fold+1}...")

    X_train_text, X_test_text = X_text_pad[train_index], X_text_pad[test_index]
    X_train_emotions, X_test_emotions = X_emotions[train_index], X_emotions[test_index]
    X_train_scores, X_test_scores = X_scores[train_index], X_scores[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = create_model()
    model.fit([X_train_text, X_train_emotions, X_train_scores], y_train, epochs=10, batch_size=32, verbose=1)


Training Fold 1...
Epoch 1/10
27/27 [==============================] - 18s 72ms/step - loss: 0.6824 - accuracy: 0.5709
Epoch 2/10
27/27 [==============================] - 2s 75ms/step - loss: 0.5298 - accuracy: 0.7459
Epoch 3/10
27/27 [==============================] - 2s 72ms/step - loss: 0.3650 - accuracy: 0.8830
Epoch 4/10
27/27 [==============================] - 2s 70ms/step - loss: 0.1621 - accuracy: 0.9610
Epoch 5/10
27/27 [==============================] - 2s 69ms/step - loss: 0.0392 - accuracy: 0.9882
Epoch 6/10
27/27 [==============================] - 2s 68ms/step - loss: 0.0078 - accuracy: 1.0000
Epoch 7/10
27/27 [==============================] - 2s 68ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 8/10
27/27 [==============================] - 2s 68ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 9/10
27/27 [==============================] - 2s 68ms/step - loss: 9.4416e-04 - accuracy: 1.0000
Epoch 10/10
27/27 [==============================] - 2s 68ms/step - loss: 7.1116e-04 

In [29]:
    loss, accuracy = model.evaluate([X_test_text, X_test_emotions, X_test_scores], y_test, batch_size=32)
    predictions = model.predict([X_test_text, X_test_emotions, X_test_scores])
    predictions = (predictions > 0.5).astype(int)


7/7 [==============================] - 3s 20ms/step


In [30]:
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

In [31]:
    tn, fp, fn, tp = tf.math.confusion_matrix(y_test, predictions).numpy().ravel()
    accuracy_scores.append((tp + tn) / (tp + tn + fp + fn))
    precision_scores.append(tp / (tp + fp))
    recall_scores.append(tp / (tp + fn))
    f1_scores.append(2 * tp / (2 * tp + fp + fn))

    print(f'Fold {fold+1} - Test Accuracy: {accuracy_scores[-1]:.4f}, Precision: {precision_scores[-1]:.4f}, Recall: {recall_scores[-1]:.4f}, F1-score: {f1_scores[-1]:.4f}')


Fold 5 - Test Accuracy: 0.7393, Precision: 0.7419, Recall: 0.8000, F1-score: 0.7699


In [32]:
print(f'\nAverage Test Accuracy: {np.mean(accuracy_scores):.4f}')
print(f'Average Precision: {np.mean(precision_scores):.4f}')
print(f'Average Recall: {np.mean(recall_scores):.4f}')
print(f'Average F1-score: {np.mean(f1_scores):.4f}')


Average Test Accuracy: 0.7393
Average Precision: 0.7419
Average Recall: 0.8000
Average F1-score: 0.7699


In [33]:
from sklearn.metrics import classification_report

In [34]:
best_fold = np.argmax(accuracy_scores)
X_train_text, X_test_text = X_text_pad[train_index], X_text_pad[test_index]
X_train_emotions, X_test_emotions = X_emotions[train_index], X_emotions[test_index]
X_train_scores, X_test_scores = X_scores[train_index], X_scores[test_index]
y_train, y_test = y[train_index], y[test_index]

best_model = create_model()
best_model.fit([X_train_text, X_train_emotions, X_train_scores], y_train, epochs=10, batch_size=32, verbose=1)


Epoch 1/10
27/27 [==============================] - 20s 138ms/step - loss: 0.6780 - accuracy: 0.5785
Epoch 2/10
27/27 [==============================] - 3s 121ms/step - loss: 0.5676 - accuracy: 0.6954
Epoch 3/10
27/27 [==============================] - 3s 104ms/step - loss: 0.2281 - accuracy: 0.9150
Epoch 4/10
27/27 [==============================] - 2s 91ms/step - loss: 0.0551 - accuracy: 0.9835
Epoch 5/10
27/27 [==============================] - 3s 94ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 6/10
27/27 [==============================] - 3s 94ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 7/10
27/27 [==============================] - 3s 94ms/step - loss: 6.6790e-04 - accuracy: 1.0000
Epoch 8/10
27/27 [==============================] - 2s 92ms/step - loss: 4.3606e-04 - accuracy: 1.0000
Epoch 9/10
27/27 [==============================] - 2s 92ms/step - loss: 3.2217e-04 - accuracy: 1.0000
Epoch 10/10
27/27 [==============================] - 2s 79ms/step - loss: 2.4175e-04 - accura

In [35]:
loss, accuracy = best_model.evaluate([X_test_text, X_test_emotions, X_test_scores], y_test, batch_size=32)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

# Make predictions
predictions = best_model.predict([X_test_text, X_test_emotions, X_test_scores])
predictions = (predictions > 0.5).astype(int)

# Print the classification report
print("\nClassification Report:")
print(classification_report(y_test, predictions))

7/7 [==============================] - 3s 47ms/step - loss: 1.2581 - accuracy: 0.7678
Test Loss: 1.2581, Test Accuracy: 0.7678
7/7 [==============================] - 2s 39ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.75      0.75        96
           1       0.79      0.78      0.79       115

    accuracy                           0.77       211
   macro avg       0.77      0.77      0.77       211
weighted avg       0.77      0.77      0.77       211



In [36]:
best_model.save('fake_news_detection_LSTM.h5')